In [1]:
import pandas as pd

In [2]:
# tx = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/LEUVEN cohort_AS.xlsx')
hla_p = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/Original/HR typing patient.xlsx')
hla_d = pd.read_excel('/Users/Danial/UMCUtrecht/Leuven/Original/HR typing donor.xlsx')

In [3]:
patient_hla_cols = ['Rec1stA', 'Rec2ndA', 'Rec1stB', 'Rec2ndB', 'Rec1stC', 'Rec2ndC', '1stDRB', '2ndDRB', '1stDRW', '2ndDRW', '1stDQB', '2ndDQB', '1stDQA', '2ndDQA']
cols = ['TX number', 'Rec ID'] + patient_hla_cols
hla_patient = hla_p[cols].assign(
    patient_hla_typing = hla_p[patient_hla_cols].apply(lambda x: ', '.join(set([hla for hla in x])), axis=1)
).drop(patient_hla_cols, axis=1)
hla_patient

,TX number,Rec ID,patient_hla_typing
0,2725,84859230,"DQB1*03:01, DRB1*11:04, B*18:03, A*03:01, B*18..."
1,2726,71420038,"DQB1*03:01, DRB1*11:01, DRB4*01:01, DRB1*04:01..."
2,2727,71612089,"DRB4*01:01, DRB3*01:01, DRB1*03:01, DQA1*02:01..."
3,2728,74510751,"DQB1*03:01, DRB1*11:04, DRB4*01:01, DQA1*02:01..."
4,2729,83270124,"DRB3*01:01, DRB1*03:01, DQB1*06:02, A*03:01, D..."
...,...,...,...
921,3857,66275884,"DQB1*03:01, B*13:02, DRB1*11:01, DQA1*02:01, A..."
922,3858,66646548,"DQB1*03:01, DRB4*01:01, DRB3*01:01, DQA1*02:01..."
923,3859,64448509,"B*40:01, A*03:01, DQA1*03:01, A*68:01, DRB1*08..."
924,3860,65415259,"DQB1*03:01, A*31:01, C*02:02, C*03:03, DRB1*11..."


In [4]:
donor_hla_cols = ['D1stA', 'D2ndA', 'D1stB', 'D2ndB', 'D1stC', 'D2ndC','1stDRB', '2ndDRB', '1stDRW', '2ndDRW', '1stDQB', '2ndDQB', '1stDQA', '2ndDQA']
cols = ['TX number', 'Don ET'] + donor_hla_cols
hla_donor = hla_d[cols].assign(
    donor_hla_typing = hla_d[donor_hla_cols].apply(lambda x: ', '.join(set([hla for hla in x])), axis=1)
).drop(donor_hla_cols, axis=1)
hla_donor

,TX number,Don ET,donor_hla_typing
0,2725,98468,"DQB1*03:01, DRB1*11:04, DRB4*01:03, A*03:01, D..."
1,2726,98494,"DQB1*03:01, DRB1*11:01, DRB1*04:01, C*07:04, D..."
2,2727,98536,"DRB4*01:01, DRB3*01:01, DRB1*03:01, DQA1*02:01..."
3,2728,98536,"DRB4*01:01, DRB3*01:01, DRB1*03:01, DQA1*02:01..."
4,2729,98671,"DRB1*04:01, DRB3*01:01, DRB1*03:01, A*03:01, D..."
...,...,...,...
921,3857,147280,"DRB4*01:01, DQA1*02:01, A*03:01, B*57:03, B*51..."
922,3858,147280,"DRB4*01:01, DQA1*02:01, A*03:01, B*57:03, B*51..."
923,3859,147279,"B*40:01, DQB1*03:01, DQB1*06:03, DQA1*01:03, D..."
924,3860,147302,"DQB1*03:01, B*38:01, C*05:01, DQB1*06:03, DRB1..."


In [5]:
hlas = hla_patient.merge(hla_donor, on='TX number')
hlas.to_csv('/Users/Danial/UMCUtrecht/Leuven/Processed/HLA Typings.csv')

# Running Antibody Pipeline:

In [10]:
from src.leuven.utility.dataclasses import (
    Transplant, 
    Participant,
    Luminex,
)
from src.leuven.antibody_analysis import AntibodyAnalysis
%load_ext autoreload
%autoreload 2

df_lsa = pd.read_csv('~/UMCUtrecht/Leuven/Processed/lsa_joined.csv')
# txs = []
ids = [
        # 2767, 
       2797, 2801, 2817, 2835, 2838, 2841, 2860, 2867, 2872, 2876, 2885, 2896, 2907, 2963, 2974, 2982, 2990, 2992, 3002, 3003, 3006,
       3026, 3042, 3075, 3104, 3105, 3111, 3118, 3135, 3140, 3152, 3179, 3188, 3211, 3218, 3229, 3265, 3272, 3279, 3288, 3295, 3296, 3312,
       3353, 3371, 3406, 3420, 3421, 3423, 3428, 3451, 3452, 3453, 3456, 3463, 3466, 3478, 3481, 3490, 3502, 3520, 3524, 3529, 3535, 3546,
       3560, 3565, 3591, 3594, 3607, 3608, 3622, 3626, 3656, 3665, 3695, 3700, 3703, 3705, 3730, 3731, 3742, 3765, 3775, 3784, 3793, 3797,
       3798, 3802, 3811, 3824, 3845, 3850, 3861
]

results = []
pipeline = AntibodyAnalysis()
hlas_dsa = hlas[hlas['TX number'].apply(lambda x: x in ids)]
for row in hlas_dsa.iterrows():
    tx_id, patient_id, patient_hla, donor_id, donor_hla = row[1]
    lsa = [Luminex(hla, assign) for assign, hla in df_lsa[df_lsa.tx_id == tx_id][['assignment', 'allele']].values]
    donor = Participant(donor_hla).get_hla(which='high_res')
    patient = Participant(patient_hla).get_hla(which='high_res')
    print(f'Donor HLA: {donor} & Patient HLA {patient}')
    tx = Transplant(
        id=tx_id, 
        donor=Participant(donor_hla), 
        recipient=Participant(patient_hla),
        lsa=lsa,
    )
    pipeline.find_desa(tx)
    pipeline.find_dsa(tx)
    results.append(pipeline.export_results())

res = pd.DataFrame(results)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Donor HLA: {'DQB1*04:02', 'C*05:01', 'DRB3*01:01', 'DRB1*03:01', 'DQA1*05:01', 'B*08:01', 'B*44:02', 'A*02:01', 'DRB1*08:01', 'DQB1*02:01', 'DQA1*04:01', 'C*07:01'} & Patient HLA {'C*03:03', 'DRB1*04:01', 'DRB3*01:01', 'DRB1*03:01', 'DQA1*03:01', 'B*08:01', 'DQA1*05:01', 'A*02:01', 'DQB1*02:01', 'DRB4*01:03', 'C*07:01', 'B*15:01', 'A*01:01', 'DQB1*03:02'}
Donor HLA {'B8', 'DR8', 'Cw5', 'B44', 'A2', 'DR17', 'DQ2', 'DQ4', 'Cw7', 'DR52'} and HLA Antibodies {'B50', 'Cw6', 'DR1403', 'B55', 'B45', 'B48', 'B37', 'DR16', 'C15', 'B67', 'B703', 'B47', 'B35', 'B56', 'B42', 'B82', 'DR51', 'C18', 'B44', 'DR7', 'B2708', 'B49', 'B53', 'B54', 'B81', 'C4', 'Cw2', 'B13', 'DR8', 'B41', 'B60', 'B7', 'A66', 'B73', 'B63', 'DR11', 'DR12', 'B61', 'B27', 'C17', 'DR13', 'Cw5', 'DR103', 'DR4', 'B78'}
Donor HLA: {'DQB1*03:01', 'DRB1*11:04', 'B*35:01', 'A*25:01', 'C*12:03', 'B*18:01', 'A*02:01', 'DQA1*05:05', 'C*04:01', 'DRB3*0

In [9]:
# res.LuminexBeads.values
# res.iloc[94, :]
res

,Epitope_Mismatch,Tx_id,Donor_HLA,Recipient_HLA,LuminexBeads,DESA_Status,DSA_Status,DESA_info,DSA_info
0,"{'56L': 'DQB1*04:02', '55R': 'DQB1*04:02', '74...",2797,"Participant(hlas=[HLA(high_res='DQB1*04:02', l...","Participant(hlas=[HLA(high_res='C*03:03', low_...","{'Positive': {'HighResolution': {'B*49:01', 'C...",DESA,DSA,"((80TLR, B*44:02), (162GLS, B*44:02), (24T, B*...","[B44, DR8, Cw5]"
1,"{'28D': 'DRB1*11:04', '85VV': 'DRB1*11:04', '5...",2801,"Participant(hlas=[HLA(high_res='DQB1*03:01', l...","Participant(hlas=[HLA(high_res='DQB1*03:01', l...","{'Positive': {'HighResolution': {'DQB1*06:03',...",DESA,DSA,"((57DE, DRB1*11:04), (11STS, DRB1*11:04), (67F...",[DR11]
2,"{'125G': 'DQB1*06:03', '87F': 'DQB1*06:03', '5...",2817,"Participant(hlas=[HLA(high_res='DQB1*06:03', l...","Participant(hlas=[HLA(high_res='DQB1*03:01', l...",{'non-Positive': {'HighResolution': {'C*02:02'...,DESA,DSA,"((166DG, A*24:02))","[B62, A24]"
3,"{'113YN': 'B*50:01', '116L': 'B*50:01', '95W':...",2835,"Participant(hlas=[HLA(high_res='DRB4*01:01', l...","Participant(hlas=[HLA(high_res='B*40:01', low_...","{'Positive': {'HighResolution': {'B*49:01', 'D...",DESA,DSA,"((73AN, C*06:02), (90D, C*06:02), (80K, C*06:0...","[B50, A32, Cw6]"
4,"{'24S': 'B*38:01', '158T': 'B*38:01', '63NI': ...",2838,"Participant(hlas=[HLA(high_res='B*38:01', low_...","Participant(hlas=[HLA(high_res='DQB1*06:03', l...","{'Positive': {'HighResolution': {'DRB1*10:01',...",DESA,DSA,"((108T, DRB5*01:01), (98KN, DRB5*01:01), (30D,...","[DQ6, DR51]"
...,...,...,...,...,...,...,...,...,...
89,"{'113YN': 'C*05:01', '177KT': 'C*05:01', '35Q'...",3811,"Participant(hlas=[HLA(high_res='DQB1*06:03', l...","Participant(hlas=[HLA(high_res='C*02:02', low_...","{'Positive': {'HighResolution': {'DPB1*01:01',...",No DESA,No DSA,(),()
90,"{'70QA': 'DRB1*04:04', '70QRA': 'DRB1*04:04', ...",3824,"Participant(hlas=[HLA(high_res='DRB1*04:04', l...","Participant(hlas=[HLA(high_res='C*16:02', low_...","{'Positive': {'HighResolution': {'A*68:02', 'A...",No DESA,DSA,(),[A2]
91,"{'52PQ': 'DQB1*05:01', '30H[DQ]': 'DQB1*05:01'...",3845,"Participant(hlas=[HLA(high_res='B*35:01', low_...","Participant(hlas=[HLA(high_res='DQB1*03:01', l...",{'non-Positive': {'HighResolution': {'DQB1*06:...,No HLAE-Abs,No HLA-Abs,(),()
92,"{'81ALR': 'B*51:01', '97T': 'B*51:01', '76EN':...",3850,"Participant(hlas=[HLA(high_res='C*01:02', low_...","Participant(hlas=[HLA(high_res='DRB4*01:01', l...","{'Positive': {'HighResolution': {'DQB1*06:03',...",DESA,DSA,"((52PQ, DQB1*05:01), (30H[DQ], DQB1*05:01), (3...",[DQ5]


In [11]:
from src.leuven.preprocess import prepare_merged_desa

desa = prepare_merged_desa(res, save_path='/Users/Danial/UMCUtrecht/Leuven/Processed/desa.csv')


In [14]:
# desa.Relevant_DESA.value_counts()

In [15]:
desa[desa.donor_LD == 1].Tx_id.unique()

array([3478, 3529, 3546, 3591, 3798])